In [4]:
# Импорт библиотек
import pandas as pd
import duckdb

In [67]:
# Задание таблиц БД
users = pd.read_csv('users.csv')
course_users = pd.read_csv('course_users.csv')
courses = pd.read_csv('courses.csv')
course_types = pd.read_csv('course_types.csv')
lessons = pd.read_csv('lessons.csv')
subjects = pd.read_csv('subjects.csv')
cities = pd.read_csv('cities.csv')
homework_done = pd.read_csv('homework_done.csv')
homework = pd.read_csv('homework.csv')
homework_lessons = pd.read_csv('homework_lessons.csv')
user_roles = pd.read_csv('user_roles.csv') 

In [68]:
query = """
SELECT 
  c.id AS 'ID курса',
  c.name AS 'Название курса',
  s.name AS 'Предмет',
  s.project AS 'Тип предмета',
  ct.name AS 'Тип курса',
  c.starts_at AS 'Дата старта курса',
  u.id AS 'ID ученика',
  u.last_name AS 'Фамилия ученика',
  ci.name AS 'Город ученика',
  cu.active AS 'Ученик не отчислен с курса',
  cu.created_at AS 'Дата открытия курса ученику',
  cu.available_lessons / c.lessons_in_month AS 'Сколько полных месяцев курса открыто у ученика',
  hdu.homework_done_by_user AS 'Число сданных ДЗ ученика на курсе'
  
FROM courses c
--При использовании джойнов использую короткие алиасы для повышения читаемости
--Присоединяем таблицу course_types с условием ct.name = 'Годовой', чтобы избежать использования подзапроса
JOIN course_types ct ON c.course_type_id = ct.id AND ct.name = 'Годовой'
LEFT JOIN subjects s ON s.id = c.subject_id
LEFT JOIN course_users cu ON c.id = cu.course_id
LEFT JOIN users u ON cu.user_id = u.id
--Присоединяем таблицу user_roles с условием ur.name = 'student', чтобы избежать использования подзапроса
LEFT JOIN user_roles ur ON u.user_role_id = ur.id AND ur.name = 'student'
LEFT JOIN cities ci ON u.city_id = ci.id
--Используем подзапрос для того, чтобы вывести число сданных домашних работ
LEFT JOIN (
    SELECT user_id, COUNT(DISTINCT homework_id) AS homework_done_by_user
    FROM homework_done
    GROUP BY user_id
) hdu ON u.id = hdu.user_id;

 """

In [69]:
# Выполнение SQL-запроса
df_result = duckdb.query(query).to_df()

In [70]:
# Вывод результата
display(df_result)

,ID курса,Название курса,Предмет,Тип предмета,Тип курса,Дата старта курса,ID ученика,Фамилия ученика,Город ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Сколько полных месяцев курса открыто у ученика,Число сданных ДЗ ученика на курсе
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10001.0,Markova,Самара,1.0,2024-07-25 18:04:20,2.0,22.0
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10001.0,Markova,Самара,1.0,2024-07-25 18:04:20,2.0,22.0
2,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10001.0,Markova,Самара,1.0,2024-09-01 07:49:22,2.0,22.0
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10006.0,Пушкина,Москва,1.0,2024-07-25 18:48:31,2.0,17.0
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10010.0,Мирзоян,Якутск,1.0,2024-07-26 11:35:32,2.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5725,84,Годовой 2к25 минимал,Литература,ЕГЭ,Годовой,2024-09-17,NaN,None,None,NaN,None,NaN,NaN
5726,87,Годовой 2к25 минимал с Катей,Математика,ЕГЭ,Годовой,2024-10-15,NaN,None,None,NaN,None,NaN,NaN
5727,99,Тестовый курс,Обществознание,ЕГЭ,Годовой,2024-10-24,NaN,None,None,NaN,None,NaN,NaN
5728,58,Годовой 2к25 платинум,История,ЕГЭ,Годовой,2024-09-18,NaN,None,None,NaN,None,NaN,NaN


In [71]:
#Проверка датасета на дубликаты
print("Количество дубликатов до удаления:", df_result.duplicated().sum())

#Удаление дубликатов
df_result.drop_duplicates(inplace=True)

print("Количество дубликатов после удаления:", df_result.duplicated().sum())

Количество дубликатов до удаления: 0
Количество дубликатов после удаления: 0


In [72]:
#Проверка и обработка пропущенных значений
print("\nПропущенные значения в каждом столбце:\n", df_result.isnull().sum())


Пропущенные значения в каждом столбце:
 ID курса                                            0
Название курса                                      0
Предмет                                             0
Тип предмета                                        0
Тип курса                                           0
Дата старта курса                                   0
ID ученика                                         12
Фамилия ученика                                    12
Город ученика                                     345
Ученик не отчислен с курса                         12
Дата открытия курса ученику                        12
Сколько полных месяцев курса открыто у ученика     27
Число сданных ДЗ ученика на курсе                 332
dtype: int64


In [73]:
#Для дальнейшего анализа есть несеолько вариантов, что делать с пропущенными значениями: удалить (строки ил столбцы), замена пропущенных значений (в самом простом случае - нулем, посложнее - средним или медианой, а также на основе соседних данных и другие более сложные варианты)
#В нашем случае самый простой и эффективный способ - удалить пропущенные значения)
df_result.dropna(inplace=True)

In [74]:
#Проверка и приведение типов данных
print("\nТипы данных до преобразования:\n", df_result.dtypes)


Типы данных до преобразования:
 ID курса                                            int64
Название курса                                     object
Предмет                                            object
Тип предмета                                       object
Тип курса                                          object
Дата старта курса                                  object
ID ученика                                        float64
Фамилия ученика                                    object
Город ученика                                      object
Ученик не отчислен с курса                        float64
Дата открытия курса ученику                        object
Сколько полных месяцев курса открыто у ученика    float64
Число сданных ДЗ ученика на курсе                 float64
dtype: object


In [75]:
#Преобразуем столбцы "Дата старта курса" и "Дата открытия курса ученику"
df_result['Дата старта курса'] = pd.to_datetime(df_result['Дата старта курса'], errors='coerce')
df_result['Дата открытия курса ученику'] = pd.to_datetime(df_result['Дата открытия курса ученику'], errors='coerce')

# Преобразование типа данных столбца из float в int
df_result['Ученик не отчислен с курса'] = df_result['Ученик не отчислен с курса'].astype(int)
df_result['Сколько полных месяцев курса открыто у ученика'] = df_result['Сколько полных месяцев курса открыто у ученика'].astype(int)
df_result['Число сданных ДЗ ученика на курсе'] = df_result['Число сданных ДЗ ученика на курсе'].astype(int)
df_result['ID ученика'] = df_result['ID ученика'].astype(int)

In [76]:
# Повторно проверяем типы данных
print("\nТипы данных после преобразования:\n", df_result.dtypes)


Типы данных после преобразования:
 ID курса                                                   int64
Название курса                                            object
Предмет                                                   object
Тип предмета                                              object
Тип курса                                                 object
Дата старта курса                                 datetime64[ns]
ID ученика                                                 int64
Фамилия ученика                                           object
Город ученика                                             object
Ученик не отчислен с курса                                 int64
Дата открытия курса ученику                       datetime64[ns]
Сколько полных месяцев курса открыто у ученика             int64
Число сданных ДЗ ученика на курсе                          int64
dtype: object


In [77]:
# Проверка на аномальные значения
# Используем описание данных для выявления аномалий
print("\nСтатистическое описание числовых столбцов:\n")
df_result.describe()


Статистическое описание числовых столбцов:



,ID курса,Дата старта курса,ID ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Сколько полных месяцев курса открыто у ученика,Число сданных ДЗ ученика на курсе
count,5096.000000,5096,5096.000000,5096.000000,5096,5096.000000,5096.000000
mean,21.151099,2024-09-19 03:03:57.362637312,24725.202904,0.966248,2024-09-18 08:02:13.720172544,2.291209,21.641680
min,16.000000,2024-09-15 00:00:00,10001.000000,0.000000,2024-07-25 18:04:20,0.000000,1.000000
25%,16.000000,2024-09-16 00:00:00,25005.750000,1.000000,2024-09-16 13:45:20,2.000000,11.000000
50%,19.000000,2024-09-17 00:00:00,28304.500000,1.000000,2024-09-25 21:09:21.500000,2.000000,18.000000
75%,23.000000,2024-09-21 00:00:00,30575.500000,1.000000,2024-10-06 14:53:15.750000128,2.000000,31.000000
max,81.000000,2024-10-15 00:00:00,33999.000000,1.000000,2024-11-09 18:44:36,10.000000,82.000000
std,7.266383,NaN,8559.682921,0.180608,NaN,1.704687,14.444967


In [78]:
# Явных аномалий не наблюдается
# Данные готовы к дальнейшему анализу
df_result.head()

,ID курса,Название курса,Предмет,Тип предмета,Тип курса,Дата старта курса,ID ученика,Фамилия ученика,Город ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Сколько полных месяцев курса открыто у ученика,Число сданных ДЗ ученика на курсе
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10001,Markova,Самара,1,2024-07-25 18:04:20,2,22
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10001,Markova,Самара,1,2024-07-25 18:04:20,2,22
2,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10001,Markova,Самара,1,2024-09-01 07:49:22,2,22
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10006,Пушкина,Москва,1,2024-07-25 18:48:31,2,17
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10010,Мирзоян,Якутск,1,2024-07-26 11:35:32,2,42


Деление учеников на волны

In [79]:
# Рассчитаем разницу в днях между датой открытия курса для ученика и датой старта курса
df_result['Дней с открытия курса'] = (df_result['Дата открытия курса ученику'] - df_result['Дата старта курса']).dt.days

In [80]:
# Создадим новый столбец 'Волна', основываясь на значении разницы в днях
def assign_wave(days):
    if days <= 0:
        return '0 волна'
    elif 1 <= days <= 7:
        return '1 волна'
    elif 8 <= days <= 14:
        return '2 волна'
    elif 15 <= days <= 21:
        return '3 волна'
    elif 22 <= days <= 28:
        return '4 волна'
    else:
        return '5 волна'

df_result['Волна'] = df_result['Дней с открытия курса'].apply(assign_wave)

In [81]:
# Посмотрим на первые строки с новым столбцом "Волна"
df_result.head()

,ID курса,Название курса,Предмет,Тип предмета,Тип курса,Дата старта курса,ID ученика,Фамилия ученика,Город ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Сколько полных месяцев курса открыто у ученика,Число сданных ДЗ ученика на курсе,Дней с открытия курса,Волна
0,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10001,Markova,Самара,1,2024-07-25 18:04:20,2,22,-53,0 волна
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10001,Markova,Самара,1,2024-07-25 18:04:20,2,22,-58,0 волна
2,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,10001,Markova,Самара,1,2024-09-01 07:49:22,2,22,-16,0 волна
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10006,Пушкина,Москва,1,2024-07-25 18:48:31,2,17,-53,0 волна
4,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10010,Мирзоян,Якутск,1,2024-07-26 11:35:32,2,42,-52,0 волна


In [82]:
#Сохраняем датасет для дальнейшего анализа
df_result.to_csv('result.csv')

In [ ]:
df_result.pivot(index=['Дата регистрации', 'Дата активности', columns='Пользователь', values='baz')